In [1]:
import torch 
from torch import nn
import torch.nn.init # for initialization
from torch.autograd import Variable

import torchvision.utils as utils
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torchvision.models as models

import matplotlib.pyplot as plt
import numpy as np

import dataset as d
import util as u
from pathlib import Path

import h5py # making dataset (or db) file maybe

In [2]:
#ImageFolder dataset --> dataset.py used folder.py obj
#all jpgs need to be wrapped by the directory of having the same number
vist_train=d.ImgDset_Folder(Path("./data/train"))  
vist_val=d.ImgDset_Folder(Path("./data/val"))  
vist_test=d.ImgDset_Folder(Path("./data/test"))  

In [12]:
len(vist_train.img_names), vist_train.img_path

(1, PosixPath('data/train'))

In [4]:
#batch_size=1 
batch_size=4
test_loader = torch.utils.data.DataLoader(dataset=vist_test,
                                          batch_size=batch_size,                 #no batch needed
                                          shuffle=False,                #no need to shuffle: feature extraction
                                          num_workers=1)

In [5]:
#model here
Resnet=models.resnet50(pretrained=True) #pretrained on "ImageNet", 
                                        #expects identically normalized/shaped inputs

In [6]:
#Resnet.cuda() #no graphic driver for desktop

In [7]:
len_test=len(vist_test)
test_name_list=vist_test.img_names

In [8]:
#test_id2idx : dict for fname to hdf5_idx
test_hdf_index=[str(i) for i in range(len_test)]
test_id2idx=dict(zip(test_name_list, test_hdf_index))


hdf=h5py.File("VIST_resnet50.hdf5", "w")
test_ds=hdf.create_dataset("VIST_test",(len_test,1000), dtype='float32' )


In [11]:
for idx,imgbatch in enumerate(test_loader):
    
    
    X=Variable(imgbatch, volatile=True) #to avoid memory prob?
    featurebatch=Resnet(X)
    print("\n\tfilename: {original} is read ".format(original=test_name_list[idx]))
    print("\tfeaturebatch shape: {fshape}".format(fshape=featurebatch.shape))
    test_ds[idx:idx+batch_size]=featurebatch.data.numpy()
    
    idx+=batch_size

hdf.close()
print("\tsaved as {hdf5}".format )



	filename: 2.png is read 
	featurebatch shape: torch.Size([4, 1000])
<built-in method format of str object at 0x7f9d9506ed20>


## 작업개요   
### 작업 시간 1+1+3시간   
>쓸데없는 제안서만 아니었어도 더 했을건데 기다리게 해서 죄송합니다ㅠ

### 만났던 난관들:   
>- dataset 어떻게 만들지? 뭔가 torchvision.dataset.ImageFolder 쓰다가 아님을 깨닫고 버림, github에서 folder.py 보고 dataset.py 구성   
- 하나씩 로드 -> feature -> hdf5 append 가 계획인데 그러는데 dataloader는 아닌거같아서...? 일단 만들고 냅둠  
- 뭔가 차원이 안맞네 np.ndarray[None,:,:,:]   

### 지금 마주한 문제:   
>- hdf5 이거 어떻게 쓰는건지 (현재)

### 해결사 킹갇전재현 찬양경배해!
>- 혼자 배우는 것도 좋지만 적절한 피드백이 배움에 큰 도움이 되는 걸 새삼 느낀다
